In [1]:
# supress warnings
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

import os
import json
from hsclient import HydroShare
from datetime import datetime
from pydantic import AnyUrl

In [2]:
# login to HydroShare
hs = HydroShare()
hs.sign_in()

resource_id = '0a7735b121f845f7ab0d1ac6348d9980' # success
# resource_id = '51957cd254b54891ba2e239428bd132d' # fails
# resource_id = '15392cb3617b4519af6ae8972f603502' # fails
hs_res = hs.resource(resource_id)

Username:  abnerbog
Password for abnerbog:  ········


In [3]:
def custom_json_encoder(obj):
    """
    Handles non-standard types like datetime, Url, and Enums for JSON serialization.
    """
    if isinstance(obj, (datetime, AnyUrl)):
        # Convert datetime objects and Url objects to ISO format strings
        return str(obj)
    # Check if the object is an Enum. Assuming your ModelProgramFileType is an Enum.
    elif hasattr(obj, 'value'): 
        return obj.value
    raise TypeError(f"Object of type {obj.__class__.__name__} is not JSON serializable")

In [4]:
# extract the resource level metaadata 
hs_res_metadata = hs_res._retrieved_metadata.model_dump()

# iterate over the model program aggregations in the resource
for agg in hs_res.aggregations(type='ModelProgram'):

    agg_metadata = agg._retrieved_metadata.model_dump()

    # add creators and published status directly from resource level metadata
    agg_metadata['resource_id'] = resource_id
    agg_metadata['creators'] = hs_res_metadata['creators']
    agg_metadata['published'] = hs_res_metadata['published'] if hs_res_metadata['published'] else None
    
    # add resource level metadata where revelevant aggregation level metadata fields are non-existant or empty
    agg_metadata['subjects'] = agg_metadata['subjects'] if agg_metadata['subjects'] else hs_res_metadata['subjects']
    agg_metadata['spatial_coverage'] = agg_metadata['spatial_coverage'] if 'spatial_coverage' in agg_metadata.keys() else hs_res_metadata['spatial_coverage']
    agg_metadata['period_coverage'] = agg_metadata['period_coverage'] if 'period_coverage' in agg_metadata.keys() else hs_res_metadata['period_coverage']
    
    # use aggreagation title as file name
    filename = f"{resource_id}-{agg_metadata['title']}.json"
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(agg_metadata, f, indent=2, ensure_ascii=False,default=custom_json_encoder)
    print(f"Saved to file: {filename}")

http://www.hydroshare.org/resource/0a7735b121f845f7ab0d1ac6348d9980/data/contents/Model Instances/System2_Model_Instances/System2_Model_Instances_resmap.xml#aggregation does not look like a valid URI, trying to serialize this will break.
http://www.hydroshare.org/resource/0a7735b121f845f7ab0d1ac6348d9980/data/contents/Model Instances/System2_Model_Instances/System2_Model_Instances_schema_values.json does not look like a valid URI, trying to serialize this will break.
http://www.hydroshare.org/resource/0a7735b121f845f7ab0d1ac6348d9980/data/contents/Model Instances/System2_Model_Instances/System2_Model_Instances_schema.json does not look like a valid URI, trying to serialize this will break.
http://www.hydroshare.org/resource/0a7735b121f845f7ab0d1ac6348d9980/data/contents/SWAT Model Executable/SWAT_COMP_resmap.xml#aggregation does not look like a valid URI, trying to serialize this will break.
http://www.hydroshare.org/resource/0a7735b121f845f7ab0d1ac6348d9980/data/contents/Model Instanc

Saved to file: 0a7735b121f845f7ab0d1ac6348d9980-SWAT 2012 Rev 664 - modified to accept long management files.json


In [5]:
agg_metadata

{'title': 'SWAT 2012 Rev 664 - modified to accept long management files',
 'subjects': ['SOIL & Water Assessment Tool', 'SWAT'],
 'language': 'eng',
 'additional_metadata': {},
 'version': 'Rev 664',
 'programming_languages': [],
 'operating_systems': ['Windows'],
 'website': Url('https://swat.tamu.edu/software/swat-executables/'),
 'file_types': [{'type': <ModelProgramFileType.software: 'https://www.hydroshare.org/terms/modelSoftware'>,
   'url': Url('http://www.hydroshare.org/resource/0a7735b121f845f7ab0d1ac6348d9980/data/contents/SWAT%20Model%20Executable/SWAT_COMP.exe')}],
 'program_schema_json': Url('http://www.hydroshare.org/resource/0a7735b121f845f7ab0d1ac6348d9980/data/contents/SWAT%20Model%20Executable/SWAT_COMP_schema.json'),
 'type': <AggregationType.ModelProgramAggregation: 'ModelProgram'>,
 'url': Url('http://www.hydroshare.org/resource/0a7735b121f845f7ab0d1ac6348d9980/data/contents/SWAT%20Model%20Executable/SWAT_COMP_resmap.xml#aggregation'),
 'rights': {'statement': 'Thi